In [1]:
from tqdm import trange
import torch

import pathpyG as pp

In [15]:
layers = 4
branches = 15

In [16]:
edges = []
prev_layer_nodes = [0]
j = 1
for _ in trange(layers):
    layer_nodes = []
    for node in prev_layer_nodes:
        for _ in range(branches):
            layer_nodes.append(j)
            edges.append((f"{node}", f"{j}"))
            j+=1
    prev_layer_nodes = layer_nodes

dag = pp.Graph.from_edge_list(edges)
dag_edge_index = dag.data.edge_index.unsqueeze(-1)

100%|██████████| 4/4 [00:00<00:00, 216.70it/s]


In [17]:
dag_edge_index

tensor([[[    0],
         [    0],
         [    0],
         ...,
         [ 3615],
         [ 3615],
         [ 3615]],

        [[    1],
         [    2],
         [    3],
         ...,
         [54238],
         [54239],
         [54240]]])

In [39]:
dag_edge_index_order_2 = pp.DAGData.lift_order_dag(dag_edge_index)

In [34]:
def lift_order_fast(edge_index):
    new_nodes = torch.cat([edge_index[0, :], edge_index[1, :, -1:]], dim=1)
    a, a_idx = edge_index[0].unique(inverse_indices=True, dim=0)
    b, b_idx = edge_index[1].unique(inverse_indices=True, dim=0)
    # intersection of a and b corresponds to all center nodes, which have
    # at least one incoming and one outgoing edge
    combined = torch.cat((a, b))
    uniques, counts = combined.unique(dim=0, return_counts=True)
    center_nodes = uniques[counts > 1]
    return new_nodes

In [35]:
lift_order_fast(dag_edge_index)

tensor([[    0,     1],
        [    0,     2],
        [    0,     3],
        ...,
        [ 3615, 54238],
        [ 3615, 54239],
        [ 3615, 54240]])

In [43]:
torch.isin(torch.tensor([[1, 2], [3, 4]]), torch.tensor([[4, 3], [1, 2]]))

tensor([[True, True],
        [True, True]])

In [47]:
dag_edge_index.view(-1, 1)

tensor([[    0],
        [    0],
        [    0],
        ...,
        [54238],
        [54239],
        [54240]])

In [62]:
edge_index = torch.tensor([[0, 1, 1, 3, 4, 1, 6],
                           [1, 2, 3, 4, 5, 6, 5]])
unique_src, src_idx = edge_index[0].unique(return_inverse=True)
unique_dst, dst_idx = edge_index[1].unique(return_inverse=True)
combined = torch.cat((unique_src, unique_dst))

In [73]:
output, inverse_indices, counts = torch.unique(combined, return_inverse=True, return_counts=True)

In [74]:
output

tensor([0, 1, 2, 3, 4, 5, 6])

In [75]:
inverse_indices.split([len(unique_src), len(unique_dst)], dim=0)

(tensor([0, 1, 3, 4, 6]), tensor([1, 2, 3, 4, 5, 6]))

In [76]:
counts

tensor([1, 2, 1, 2, 2, 1, 2])